In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

%matplotlib inline
%load_ext autoreload
%autoreload 2

Set up training data

In the next cell, we set up the training data for this example. We’ll be using 100 regularly spaced points on [0,1] which we evaluate the function on and add Gaussian noise to get the training labels.

In [2]:
filename = "/Users/litz/Documents/GitHub/learning_GPyTorch/df.csv"
df = pd.read_csv(filename, index_col=None, header=0)
df

Unnamed: 0  evol_stage  initial_mass  initial_Yinit  initial_feh  \
0               0           1          1.05           0.28          0.0   
1               1           1          1.05           0.28          0.0   
2               2           1          1.05           0.28          0.0   
3               3           1          1.05           0.28          0.0   
4               4           1          1.05           0.28          0.0   
...           ...         ...           ...            ...          ...   
19440       19440           2          0.98           0.28          0.0   
19441       19441           2          0.98           0.28          0.0   
19442       19442           2          0.98           0.28          0.0   
19443       19443           2          0.98           0.28          0.0   
19444       19444           2          0.98           0.28          0.0   

       initial_MLT   star_age   frac_age    ms_age  effective_T     log_g  \
0              2.1   0.089514   1.025962  5.701476  6020.401864  4.487184   
1              2.1   0.102056   1.029653  5.701476  6021.004397  4.486433   
2              2.1   0.117105   1.034099  5.701476  6021.569058  4.485608   
3              2.1   0.135165   1.039460  5.701476  6022.148049  4.484698   
4              2.1   0.156836   1.045930  5.701476  6022.769456  4.483688   
...            ...        ...        ...       ...          ...       ...   
19440          2.1  10.370700   9.975236  7.729887  4991.771740  3.614765   
19441          2.1  10.374359   9.983335  7.729887  4990.137910  3.610087   
19442          2.1  10.376164   9.987332  7.729887  4989.380370  3.607805   
19443          2.1  10.379737   9.995249  7.729887  4987.917756  3.603286   
19444          2.1  10.381880  10.000000  7.729887  4987.002476  3.600491   

         radius  star_feh  delta_nu_fit       nu_max  hrgradient  fagegradient  
0      0.968160  0.998265    145.333042  3401.690893    0.000197      0.002016  
1      0.968998  0.997933    145.148810  3395.642769    0.000204      0.002420  
2      0.969919  0.997538    144.946449  3389.036269    0.000219      0.002904  
3      0.970936  0.997065    144.723391  3381.780702    0.000240      0.003484  
4      0.972065  0.996500    144.476115  3373.756599    0.000267      0.004181  
...         ...       ...           ...          ...         ...           ...  
19440  2.549161  0.990766     32.780509   501.142795    0.000530      0.000237  
19441  2.562915  0.990908     32.509757   495.854066    0.000545      0.000235  
19442  2.569650  0.991016     32.381770   493.292949    0.000529      0.000232  
19443  2.583039  0.991272     32.125098   488.258024    0.000585      0.000254  
19444  2.591353  0.991346     31.968747   485.171211    0.000708      0.000305  

[19445 rows x 17 columns]

In [3]:
masses = np.unique(df.initial_mass)

mm = 1.10

dfu = df[df.initial_mass == 1.10]

In [4]:
train_x = torch.tensor(list(dfu["frac_age"]))
train_y = torch.tensor(list((dfu["effective_T"])))
plt.scatter(train_x, train_y)
torch.is_tensor(train_x)
torch.is_tensor(train_y)

True

Setting up the model

The next cell demonstrates the most critical features of a user-defined Gaussian process model in GPyTorch. Building a GP model in GPyTorch is different in a number of ways.

First in contrast to many existing GP packages, we do not provide full GP models for the user. Rather, we provide the tools necessary to quickly construct one. This is because we believe, analogous to building a neural network in standard PyTorch, it is important to have the flexibility to include whatever components are necessary. As can be seen in more complicated examples, this allows the user great flexibility in designing custom models.

For most GP regression models, you will need to construct the following GPyTorch objects:

A GP Model (gpytorch.models.ExactGP) - This handles most of the inference.
A Likelihood (gpytorch.likelihoods.GaussianLikelihood) - This is the most common likelihood used for GP regression.
A Mean - This defines the prior mean of the GP.(If you don’t know which mean to use, a gpytorch.means.ConstantMean() is a good place to start.)
A Kernel - This defines the prior covariance of the GP.(If you don’t know which kernel to use, a gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()) is a good place to start).
A MultivariateNormal Distribution (gpytorch.distributions.MultivariateNormal) - This is the object used to represent multivariate normal distributions.
The GP Model

The components of a user built (Exact, i.e. non-variational) GP model in GPyTorch are, broadly speaking:

An __init__ method that takes the training data and a likelihood, and constructs whatever objects are necessary for the model’s forward method. This will most commonly include things like a mean module and a kernel module.
A forward method that takes in some 𝑛×𝑑 data x and returns a MultivariateNormal with the prior mean and covariance evaluated at x. In other words, we return the vector 𝜇(𝑥) and the 𝑛×𝑛 matrix 𝐾𝑥𝑥 representing the prior mean and covariance matrix of the GP.
This specification leaves a large amount of flexibility when defining a model. For example, to compose two kernels via addition, you can either add the kernel modules directly:

self.covar_module = ScaleKernel(RBFKernel() + WhiteNoiseKernel())

Or you can add the outputs of the kernel in the forward method:

covar_x = self.rbf_kernel_module(x) + self.white_noise_module(x)

In [12]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

Model modes

Like most PyTorch modules, the ExactGP has a .train() and .eval() mode. - .train() mode is for optimizing model hyperameters. - .eval() mode is for computing predictions through the model posterior.

Training the model

In the next cell, we handle using Type-II MLE to train the hyperparameters of the Gaussian process.

The most obvious difference here compared to many other GP implementations is that, as in standard PyTorch, the core training loop is written by the user. In GPyTorch, we make use of the standard PyTorch optimizers as from torch.optim, and all trainable parameters of the model should be of type torch.nn.Parameter. Because GP models directly extend torch.nn.Module, calls to methods like model.parameters() or model.named_parameters() function as you might expect coming from PyTorch.

In most cases, the boilerplate code below will work well. It has the same basic components as the standard PyTorch training loop:

Zero all parameter gradients
Call the model and compute the loss
Call backward on the loss to fill in gradients
Take a step on the optimizer
However, defining custom training loops allows for greater flexibility. For example, it is easy to save the parameters at each step of training, or use different learning rates for different parameters (which may be useful in deep kernel learning for example).

In [21]:
def model_run(step = None):
    # this is for running the notebook in our testing framework
    smoke_test = ('CI' in os.environ)
    training_iter = 2 if smoke_test else step


    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam([
        {'params': model.parameters()},  # Includes GaussianLikelihood parameters
    ], lr=0.1)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter): 
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
            i + 1, training_iter, loss.item(),
            model.covar_module.base_kernel.lengthscale.item(),
            model.likelihood.noise.item()
        ))
        optimizer.step()
    return

Make predictions with the model

In the next cell, we make predictions with the model. To do this, we simply put the model and likelihood in eval mode, and call both modules on the test data.

Just as a user defined GP model returns a MultivariateNormal containing the prior mean and covariance from forward, a trained GP model in eval mode returns a MultivariateNormal containing the posterior mean and covariance. Thus, getting the predictive mean and variance, and then sampling functions from the GP at the given test points could be accomplished with calls like:

f_preds = model(test_x)
y_preds = likelihood(model(test_x))

f_mean = f_preds.mean
f_var = f_preds.variance
f_covar = f_preds.covariance_matrix
f_samples = f_preds.sample(sample_shape=torch.Size(1000,))

The gpytorch.settings.fast_pred_var context is not needed, but here we are giving a preview of using one of our cool features, getting faster predictive distributions using LOVE.

Plot the model fit

In the next cell, we plot the mean and confidence region of the Gaussian process model. The confidence_region method is a helper method that returns 2 standard deviations above and below the mean.

In [ ]:
model_run(step = 5000)

Iter 1/5000 - Loss: 209.532   lengthscale: 11.509   noise: 196.129
Iter 2/5000 - Loss: 209.426   lengthscale: 11.409   noise: 196.229
Iter 3/5000 - Loss: 209.304   lengthscale: 11.479   noise: 196.329
Iter 4/5000 - Loss: 209.191   lengthscale: 11.547   noise: 196.429
Iter 5/5000 - Loss: 209.080   lengthscale: 11.573   noise: 196.529
Iter 6/5000 - Loss: 208.964   lengthscale: 11.563   noise: 196.629
Iter 7/5000 - Loss: 208.848   lengthscale: 11.532   noise: 196.729
Iter 8/5000 - Loss: 208.733   lengthscale: 11.496   noise: 196.829
Iter 9/5000 - Loss: 208.621   lengthscale: 11.466   noise: 196.929
Iter 10/5000 - Loss: 208.509   lengthscale: 11.452   noise: 197.028
Iter 11/5000 - Loss: 208.395   lengthscale: 11.455   noise: 197.128
Iter 12/5000 - Loss: 208.281   lengthscale: 11.472   noise: 197.228
Iter 13/5000 - Loss: 208.167   lengthscale: 11.496   noise: 197.328
Iter 14/5000 - Loss: 208.054   lengthscale: 11.518   noise: 197.428
Iter 15/5000 - Loss: 207.942   lengthscale: 11.534   nois

Iter 145/5000 - Loss: 194.592   lengthscale: 11.486   noise: 210.144
Iter 146/5000 - Loss: 194.498   lengthscale: 11.486   noise: 210.238
Iter 147/5000 - Loss: 194.405   lengthscale: 11.486   noise: 210.332
Iter 148/5000 - Loss: 194.312   lengthscale: 11.486   noise: 210.426
Iter 149/5000 - Loss: 194.219   lengthscale: 11.486   noise: 210.520
Iter 150/5000 - Loss: 194.126   lengthscale: 11.486   noise: 210.614
Iter 151/5000 - Loss: 194.034   lengthscale: 11.486   noise: 210.708
Iter 152/5000 - Loss: 193.941   lengthscale: 11.486   noise: 210.802
Iter 153/5000 - Loss: 193.848   lengthscale: 11.486   noise: 210.896
Iter 154/5000 - Loss: 193.756   lengthscale: 11.485   noise: 210.990
Iter 155/5000 - Loss: 193.664   lengthscale: 11.485   noise: 211.084
Iter 156/5000 - Loss: 193.572   lengthscale: 11.485   noise: 211.177
Iter 157/5000 - Loss: 193.480   lengthscale: 11.485   noise: 211.271
Iter 158/5000 - Loss: 193.388   lengthscale: 11.485   noise: 211.365
Iter 159/5000 - Loss: 193.296   le

Iter 284/5000 - Loss: 182.717   lengthscale: 11.471   noise: 222.862
Iter 285/5000 - Loss: 182.638   lengthscale: 11.471   noise: 222.951
Iter 286/5000 - Loss: 182.560   lengthscale: 11.471   noise: 223.040
Iter 287/5000 - Loss: 182.482   lengthscale: 11.471   noise: 223.129
Iter 288/5000 - Loss: 182.404   lengthscale: 11.471   noise: 223.217
Iter 289/5000 - Loss: 182.327   lengthscale: 11.471   noise: 223.306
Iter 290/5000 - Loss: 182.249   lengthscale: 11.471   noise: 223.395
Iter 291/5000 - Loss: 182.171   lengthscale: 11.471   noise: 223.484
Iter 292/5000 - Loss: 182.094   lengthscale: 11.470   noise: 223.573
Iter 293/5000 - Loss: 182.016   lengthscale: 11.470   noise: 223.661
Iter 294/5000 - Loss: 181.939   lengthscale: 11.470   noise: 223.750
Iter 295/5000 - Loss: 181.862   lengthscale: 11.470   noise: 223.839
Iter 296/5000 - Loss: 181.785   lengthscale: 11.470   noise: 223.927
Iter 297/5000 - Loss: 181.707   lengthscale: 11.470   noise: 224.016
Iter 298/5000 - Loss: 181.630   le

Iter 420/5000 - Loss: 172.868   lengthscale: 11.457   noise: 234.680
Iter 421/5000 - Loss: 172.801   lengthscale: 11.457   noise: 234.765
Iter 422/5000 - Loss: 172.734   lengthscale: 11.457   noise: 234.850
Iter 423/5000 - Loss: 172.667   lengthscale: 11.457   noise: 234.935
Iter 424/5000 - Loss: 172.600   lengthscale: 11.457   noise: 235.020
Iter 425/5000 - Loss: 172.533   lengthscale: 11.457   noise: 235.104
Iter 426/5000 - Loss: 172.466   lengthscale: 11.456   noise: 235.189
Iter 427/5000 - Loss: 172.400   lengthscale: 11.456   noise: 235.274
Iter 428/5000 - Loss: 172.333   lengthscale: 11.456   noise: 235.359
Iter 429/5000 - Loss: 172.267   lengthscale: 11.456   noise: 235.443
Iter 430/5000 - Loss: 172.200   lengthscale: 11.456   noise: 235.528
Iter 431/5000 - Loss: 172.134   lengthscale: 11.456   noise: 235.613
Iter 432/5000 - Loss: 172.068   lengthscale: 11.456   noise: 235.697
Iter 433/5000 - Loss: 172.001   lengthscale: 11.456   noise: 235.782
Iter 434/5000 - Loss: 171.935   le

Iter 561/5000 - Loss: 164.056   lengthscale: 11.443   noise: 246.401
Iter 562/5000 - Loss: 163.998   lengthscale: 11.442   noise: 246.483
Iter 563/5000 - Loss: 163.939   lengthscale: 11.442   noise: 246.564
Iter 564/5000 - Loss: 163.881   lengthscale: 11.442   noise: 246.645
Iter 565/5000 - Loss: 163.823   lengthscale: 11.442   noise: 246.727
Iter 566/5000 - Loss: 163.765   lengthscale: 11.442   noise: 246.808
Iter 567/5000 - Loss: 163.707   lengthscale: 11.442   noise: 246.889
Iter 568/5000 - Loss: 163.649   lengthscale: 11.442   noise: 246.971
Iter 569/5000 - Loss: 163.591   lengthscale: 11.442   noise: 247.052
Iter 570/5000 - Loss: 163.534   lengthscale: 11.442   noise: 247.133
Iter 571/5000 - Loss: 163.476   lengthscale: 11.442   noise: 247.215
Iter 572/5000 - Loss: 163.418   lengthscale: 11.441   noise: 247.296
Iter 573/5000 - Loss: 163.361   lengthscale: 11.441   noise: 247.377
Iter 574/5000 - Loss: 163.303   lengthscale: 11.441   noise: 247.458
Iter 575/5000 - Loss: 163.246   le

...

Iter 3721/5000 - Loss: 82.695   lengthscale: 11.147   noise: 460.650
Iter 3722/5000 - Loss: 82.682   lengthscale: 11.148   noise: 460.715
Iter 3723/5000 - Loss: 82.669   lengthscale: 11.147   noise: 460.779
Iter 3724/5000 - Loss: 82.656   lengthscale: 11.148   noise: 460.843
Iter 3725/5000 - Loss: 82.643   lengthscale: 11.147   noise: 460.907
Iter 3726/5000 - Loss: 82.630   lengthscale: 11.148   noise: 460.972
Iter 3727/5000 - Loss: 82.617   lengthscale: 11.146   noise: 461.036
Iter 3728/5000 - Loss: 82.604   lengthscale: 11.148   noise: 461.100
Iter 3729/5000 - Loss: 82.591   lengthscale: 11.146   noise: 461.164
Iter 3730/5000 - Loss: 82.578   lengthscale: 11.148   noise: 461.229
Iter 3731/5000 - Loss: 82.566   lengthscale: 11.145   noise: 461.293
Iter 3732/5000 - Loss: 82.553   lengthscale: 11.149   noise: 461.357
Iter 3733/5000 - Loss: 82.540   lengthscale: 11.144   noise: 461.421
Iter 3734/5000 - Loss: 82.527   lengthscale: 11.149   noise: 461.486
Iter 3735/5000 - Loss: 82.514   le

Iter 3853/5000 - Loss: 81.016   lengthscale: 11.136   noise: 469.141
Iter 3854/5000 - Loss: 81.003   lengthscale: 11.136   noise: 469.206
Iter 3855/5000 - Loss: 80.991   lengthscale: 11.135   noise: 469.270
Iter 3856/5000 - Loss: 80.978   lengthscale: 11.135   noise: 469.334
Iter 3857/5000 - Loss: 80.966   lengthscale: 11.135   noise: 469.399
Iter 3858/5000 - Loss: 80.953   lengthscale: 11.135   noise: 469.463
Iter 3859/5000 - Loss: 80.941   lengthscale: 11.135   noise: 469.528
Iter 3860/5000 - Loss: 80.928   lengthscale: 11.135   noise: 469.592
Iter 3861/5000 - Loss: 80.916   lengthscale: 11.135   noise: 469.656
Iter 3862/5000 - Loss: 80.903   lengthscale: 11.135   noise: 469.721
Iter 3863/5000 - Loss: 80.891   lengthscale: 11.135   noise: 469.785
Iter 3864/5000 - Loss: 80.878   lengthscale: 11.135   noise: 469.850
Iter 3865/5000 - Loss: 80.866   lengthscale: 11.135   noise: 469.914
Iter 3866/5000 - Loss: 80.853   lengthscale: 11.134   noise: 469.978
Iter 3867/5000 - Loss: 80.841   le

Iter 3984/5000 - Loss: 79.406   lengthscale: 11.123   noise: 477.586
Iter 3985/5000 - Loss: 79.394   lengthscale: 11.124   noise: 477.650
Iter 3986/5000 - Loss: 79.382   lengthscale: 11.123   noise: 477.715
Iter 3987/5000 - Loss: 79.370   lengthscale: 11.123   noise: 477.779
Iter 3988/5000 - Loss: 79.358   lengthscale: 11.123   noise: 477.844
Iter 3989/5000 - Loss: 79.346   lengthscale: 11.123   noise: 477.909
Iter 3990/5000 - Loss: 79.333   lengthscale: 11.123   noise: 477.973
Iter 3991/5000 - Loss: 79.321   lengthscale: 11.123   noise: 478.038
Iter 3992/5000 - Loss: 79.309   lengthscale: 11.123   noise: 478.102
Iter 3993/5000 - Loss: 79.297   lengthscale: 11.122   noise: 478.167
Iter 3994/5000 - Loss: 79.285   lengthscale: 11.123   noise: 478.231
Iter 3995/5000 - Loss: 79.273   lengthscale: 11.122   noise: 478.296
Iter 3996/5000 - Loss: 79.261   lengthscale: 11.123   noise: 478.360
Iter 3997/5000 - Loss: 79.249   lengthscale: 11.122   noise: 478.425
Iter 3998/5000 - Loss: 79.237   le

Iter 4115/5000 - Loss: 77.849   lengthscale: 11.111   noise: 486.051
Iter 4116/5000 - Loss: 77.838   lengthscale: 11.113   noise: 486.116
Iter 4117/5000 - Loss: 77.826   lengthscale: 11.109   noise: 486.180
Iter 4118/5000 - Loss: 77.814   lengthscale: 11.114   noise: 486.245
Iter 4119/5000 - Loss: 77.803   lengthscale: 11.108   noise: 486.310
Iter 4120/5000 - Loss: 77.791   lengthscale: 11.114   noise: 486.374
Iter 4121/5000 - Loss: 77.779   lengthscale: 11.108   noise: 486.439
Iter 4122/5000 - Loss: 77.768   lengthscale: 11.112   noise: 486.504
Iter 4123/5000 - Loss: 77.756   lengthscale: 11.110   noise: 486.569
Iter 4124/5000 - Loss: 77.744   lengthscale: 11.110   noise: 486.633
Iter 4125/5000 - Loss: 77.733   lengthscale: 11.111   noise: 486.698
Iter 4126/5000 - Loss: 77.721   lengthscale: 11.109   noise: 486.763
Iter 4127/5000 - Loss: 77.709   lengthscale: 11.112   noise: 486.828
Iter 4128/5000 - Loss: 77.698   lengthscale: 11.108   noise: 486.892
Iter 4129/5000 - Loss: 77.686   le

Iter 4253/5000 - Loss: 76.264   lengthscale: 11.098   noise: 494.993
Iter 4254/5000 - Loss: 76.253   lengthscale: 11.099   noise: 495.058
Iter 4255/5000 - Loss: 76.241   lengthscale: 11.097   noise: 495.123
Iter 4256/5000 - Loss: 76.230   lengthscale: 11.100   noise: 495.188
Iter 4257/5000 - Loss: 76.219   lengthscale: 11.096   noise: 495.253
Iter 4258/5000 - Loss: 76.208   lengthscale: 11.100   noise: 495.318
Iter 4259/5000 - Loss: 76.196   lengthscale: 11.095   noise: 495.383
Iter 4260/5000 - Loss: 76.185   lengthscale: 11.101   noise: 495.448
Iter 4261/5000 - Loss: 76.174   lengthscale: 11.094   noise: 495.513
Iter 4262/5000 - Loss: 76.163   lengthscale: 11.102   noise: 495.578
Iter 4263/5000 - Loss: 76.151   lengthscale: 11.092   noise: 495.643
Iter 4264/5000 - Loss: 76.140   lengthscale: 11.104   noise: 495.708
Iter 4265/5000 - Loss: 76.129   lengthscale: 11.089   noise: 495.772
Iter 4266/5000 - Loss: 76.118   lengthscale: 11.107   noise: 495.837
Iter 4267/5000 - Loss: 76.106   le

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x = train_x
    observed_pred = likelihood(model(test_x))
    
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(10, 9))

    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), observed_pred.mean.numpy(), 'r')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    #ax.set_xlim([3.9,3.6])
    #ax.set_ylim([-.5, 2])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])